In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error


In [20]:
merged_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Walmart Time Series /dataset/merged_data_after_FE.csv')
merged_data

,Store,Dept,Weekly_Sales,IsHoliday_x,Fuel_Price,IsHoliday_y,PreHoliday,PostHoliday,CPI_standardized,Temperature_standardized,...,Lag_1_Week,Lag_4_Weeks,Lag_12_Weeks,Rolling_4_Week_Avg,Normalized_Size,Type_B,Type_C,Weeks_Since_Holiday,Seasonal_Component,Residual_Component
0,1,1,0.042851,0,0.050100,0,1,0,1.018773,-0.963797,...,0.030040,0.030040,0.030040,0.049846,0.239209,False,False,0.015873,0.425080,0.126605
1,1,1,0.073097,1,0.038076,1,0,0,1.022497,-1.169782,...,0.042851,0.030040,0.030040,0.049846,0.239209,False,False,0.000000,0.875189,0.126605
2,1,1,0.066732,0,0.021042,0,0,1,1.023696,-1.092809,...,0.073097,0.030040,0.030040,0.049846,0.239209,False,False,0.015873,0.604729,0.126605
3,1,1,0.034942,0,0.044589,0,0,0,1.024475,-0.729624,...,0.066732,0.030040,0.030040,0.099806,0.239209,False,False,0.031746,0.153977,0.126605
4,1,1,0.038415,0,0.076653,0,0,0,1.025254,-0.736671,...,0.034942,0.042851,0.030040,0.097532,0.239209,False,False,0.047619,0.301357,0.126605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,0.007875,0,0.764028,0,0,0,0.531461,0.259647,...,0.007816,0.007642,0.008092,0.004327,-0.303489,True,False,0.047619,0.875189,0.126445
421566,45,98,0.008046,0,0.758016,0,0,0,0.535466,0.260189,...,0.007875,0.007651,0.008142,0.004529,-0.303489,True,False,0.063492,0.604729,0.126445
421567,45,98,0.008666,0,0.765531,0,0,0,0.539472,-0.304644,...,0.008046,0.008015,0.008357,0.004863,-0.303489,True,False,0.079365,0.153977,0.126445
421568,45,98,0.008235,0,0.750000,0,0,0,0.539563,-0.196231,...,0.008666,0.007816,0.008089,0.005078,-0.303489,True,False,0.095238,0.301357,0.126445


In [3]:
# # Install RAPIDS using the script provided for Colab compatibility
!pip install -q condacolab
import condacolab
condacolab.install()

# Install RAPIDS 23.02 for CUDA 11 (compatible with Colab’s GPUs)
!conda install -y -c rapidsai -c nvidia -c conda-forge \
    cudf=23.02 cuml=23.02 cugraph=23.02


Streaming output truncated to the last 5000 lines.






libcusparse-11.7.5.8 | 176.3 MB  | :  85% 0.8487444000043594/1 [00:45<00:00,  5.79s/it]







libcugraphops-23.02. | 224.9 MB  | :  98% 0.9846274778939571/1 [00:45<00:00,  7.44s/it]








libcusparse-11.7.5.8 | 176.3 MB  | :  87% 0.8661127339363759/1 [00:46<00:00,  7.38s/it]







libcugraphops-23.02. | 224.9 MB  | : 100% 0.9983115423133653/1 [00:46<00:00,  8.56s/it]








libcusparse-11.7.5.8 | 176.3 MB  | :  88% 0.880911263358043/1 [00:46<00:00,  7.45s/it] 








libcusparse-11.7.5.8 | 176.3 MB  | :  90% 0.8951781090879137/1 [00:46<00:00,  7.89s/it]








libcusparse-11.7.5.8 | 176.3 MB  | :  91% 0.9147617917357487/1 [00:46<00:00,  7.46s/it]








libcusparse-11.7.5.8 | 176.3 MB  | :  93% 0.9317756698732343/1 [00:46<00:00,  7.08s/it]








libcusparse-11.7.5.8 | 176.3 MB  | :  95% 0.9511821246238038/1 [00:46<00:00,  6.47s/it]








libcusparse-11.7.5.8 | 176.3 MB  | :  97% 0.9692593701448823/1 [00:46<00:00,  6.4

In [1]:
# Assuming X_train, X_test, y_train, y_test are your train-test splits in Pandas
import cudf


/usr/local/lib/python3.10/site-packages/cupy/_environment.py:440: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [21]:

# Define the target and feature variables
X = merged_data.drop(columns=['Weekly_Sales'])  # All columns except the target
y = merged_data['Weekly_Sales']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the Pandas DataFrames to cuDF DataFrames
X_train_cudf = cudf.DataFrame.from_pandas(x_train)
X_test_cudf = cudf.DataFrame.from_pandas(x_test)
y_train_cudf = cudf.Series(y_train)
y_test_cudf = cudf.Series(y_test)


In [22]:
import cuml

In [25]:
from cuml.linear_model import LinearRegression

# Initialize and train cuML Linear Regression
cuml_lr = LinearRegression()
cuml_lr.fit(X_train_cudf, y_train_cudf)

# Predict and evaluate
y_pred_cuml_lr = cuml_lr.predict(X_test_cudf).to_pandas()
mse_cuml_lr = mean_squared_error(y_test_cudf.to_pandas(), y_pred_cuml_lr)
print(f"cuML Linear Regression MSE: {mse_cuml_lr}")


cuML Linear Regression MSE: 2.9246690949054185e-05


In [26]:

from cuml.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# # Initialize and train the cuML Random Forest model
cuml_rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
cuml_rf.fit(X_train_cudf, y_train_cudf)

# Predict on the test set
y_pred_cuml_rf = cuml_rf.predict(X_test_cudf).to_pandas()  # Convert predictions back to Pandas

# Evaluate the model
mse_cuml_rf = mean_squared_error(y_test_cudf.to_pandas(), y_pred_cuml_rf)
print(f"cuML Random Forest MSE: {mse_cuml_rf}")


/usr/local/lib/python3.10/site-packages/cuml/internals/api_decorators.py:342: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/usr/local/lib/python3.10/site-packages/cuml/internals/api_decorators.py:190: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


cuML Random Forest MSE: 1.91346098918556e-05


In [27]:
from xgboost import XGBRFRegressor
from sklearn.metrics import mean_squared_error

# Initialize XGBRFRegressor with GPU support
xgbrf_model = XGBRFRegressor(n_estimators=100, tree_method='gpu_hist', random_state=42)

# Train the model
xgbrf_model.fit(x_train, y_train)

# Predict and evaluate
y_pred = xgbrf_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print(f"XGBRFRegressor MSE: {mse}")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:19:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGBRFRegressor MSE: 3.016293744881835e-05


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:19:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [07:19:14] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
